### Задание 1



Сбор данных из ИСУ.

Выберите какой-либо факультет, соберите данные по обучающимся на нём группам бакалавров на всех курсах (ИСУ - Учебный процесс - Учебные группы и потоки). Сформируйте таблицу с колонками `группа`, `курс`, `номер_ису`, `фио`. Ответьте на вопросы:

1. Одинаковое ли количество групп от курса к курсу?

2. Есть ли в вашей выборке однофамильцы (для простоты учитывайте только полное совпадение фамилии)? Если да, выведите этих людей.

3. Есть ли в вашей выборке ребята без отчества? Если да, выведите этих людей.

4. Найдите курс, на котором меньше всего обучающихся. Выведите значение и номер курса. Если таких курсов несколько, выведите их все.

5. Для каждого курса найдите группу, в которой больше всего обучающихся. Выведите тройки (курс, группа, кол-во обучающихся). Если для какого-либо курса таких групп окажется несколько, выведите их все.

Старайтесь писать код так, чтобы при изменении наименования факультета ничего не ломалось.

In [ ]:
# Это необходимо, чтобы запустить код в Google Colab. Источник: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
%%capture
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver-autoinstaller

In [ ]:
import sys
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller


sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--verbose')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920, 1200')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)  # создание объекта драйвера

In [ ]:
# вход в ИСУ
driver.get("https://id.itmo.ru/login")

# подождать, пока подгрузится страница авторизации
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "username"))  # если случится TimeoutException - перезапустите ячейку
)

<selenium.webdriver.remote.webelement.WebElement (session="2d475f1204da464812d9aca5d9e42dca", element="f.5EACB7E3492367E23BC53DAC710CEFE6.d.1FF58DC67A91117434A3F6B4D8FEB050.e.11")>

In [ ]:
# информация для входа в ИСУ
with open('/content/creds.txt') as f:
  creds = f.read().split()
  username = creds[0]
  password = creds[1]

# ввод данных для входа
username_field = driver.find_element(By.NAME, "username")
password_field = driver.find_element(By.NAME, "password")

username_field.send_keys(username)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)



In [ ]:
target_faculty_name = 'факультет инфокоммуникационных технологий'

In [ ]:

driver.get('https://isu.ifmo.ru/pls/apex/f?p=2143:9:111405407664066::NO:RP:')

wait_for_cmd_f = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Для поиска нужной группы')]"))
)


soup = BeautifulSoup(driver.page_source, 'html.parser')
page_contents = soup.text
page_contents = page_contents[page_contents.find('Cmd+F.') + len('Cmd+F.'):].replace('  ', '').replace('\n\n', '')


In [ ]:
bakalavriat = 'Бакалавриат'
magistratura = 'Магистратура'
'''specialitet = 'Специалитет'
aspirantura = 'Аспирантура'
doktorantura = 'Докторантура'
drugie = 'Другие'''

INDEX_BAK = page_contents.find(bakalavriat) + len(bakalavriat)
INDEX_MAG = page_contents.find(magistratura)
'''INDEX_SPEC = page_contents.find() + len()
INDEX_ASP = page_contents.find() + len()
INDEX_DOK = page_contents.find() + len()
INDEX_DRUG = page_contents.find() + len()'''

'INDEX_SPEC = page_contents.find() + len()\nINDEX_ASP = page_contents.find() + len()\nINDEX_DOK = page_contents.find() + len()\nINDEX_DRUG = page_contents.find() + len()'

In [ ]:
page_contents = page_contents[INDEX_BAK:INDEX_MAG].strip()

In [ ]:
for i in range(1, 5):
  page_contents = page_contents.replace(f'{i}-й курс', '')

In [ ]:
page_contents = page_contents.split('\n')
page_contents = [i.strip() for i in page_contents]

In [ ]:
faculties = []

In [ ]:
for i in page_contents:
  if ' ' in i and 'курс' not in i:
    faculties.append(i)

In [ ]:
faculties = list(set(faculties))

In [ ]:
print(faculties)

['факультет инфокоммуникационных технологий', 'международный научно-образовательный центр физики наноструктур', 'центр химической инженерии', 'факультет экотехнологий', 'научно-образовательный центр фотоники и оптоинформатики', 'образовательный центр "Энергоэффективные инженерные системы"', 'факультет биотехнологий', 'факультет систем управления и робототехники', 'центр прикладной оптики', 'физический факультет', 'факультет безопасности информационных технологий', 'институт международного развития и партнерства', 'факультет программной инженерии и компьютерной техники', 'школа разработки видеоигр', 'институт перспективных систем передачи данных', 'научно-образовательный центр инфохимии', 'факультет информационных технологий и программирования', 'институт лазерных технологий', 'факультет технологического менеджмента и инноваций']


In [ ]:
first_course = page_contents[page_contents.index('1 курс') + 1: page_contents.index('2 курс')]
second_course = page_contents[page_contents.index('2 курс') + 1: page_contents.index('3 курс')]
third_course = page_contents[page_contents.index('3 курс') + 1: page_contents.index('4 курс')]
fourth_course = page_contents[page_contents.index('4 курс') + 1:]

In [ ]:
print(first_course)
print(second_course)
print(third_course)
print(fourth_course)

['институт лазерных технологий', 'L3116', 'L3117', 'L3118', 'институт международного развития и партнерства', 'D3110', 'институт перспективных систем передачи данных', 'D3121c', 'D3130c', 'международный научно-образовательный центр физики наноструктур', 'V3145', 'научно-образовательный центр инфохимии', 'O3143', 'O3144', 'научно-образовательный центр фотоники и оптоинформатики', 'V3102', 'факультет безопасности информационных технологий', 'N3145', 'N3146', 'N3147', 'N3148', 'N3149', 'N3150', 'N3151', 'факультет биотехнологий', 'T3130', 'T3131', 'T3132', 'факультет инфокоммуникационных технологий', 'K3120', 'K3121', 'K3122', 'K3123', 'K3139', 'K3140', 'K3141', 'K3142', 'K3143', 'K3144', 'K3150', 'факультет информационных технологий и программирования', 'M3100', 'M3101', 'M3102', 'M3103', 'M3104', 'M3105', 'M3106', 'M3107', 'M3108', 'M3109', 'M3110', 'M3111', 'M3112', 'M3113', 'M3114', 'M3115', 'M3116', 'M3117', 'M3118', 'M3119', 'M3132', 'M3133', 'M3134', 'M3135', 'M3136', 'M3137', 'M31

In [ ]:
isu_data = pd.DataFrame(columns=['group', 'course', 'faculty'])

In [ ]:
page_contents_upper = [i.upper() for i in page_contents]

In [ ]:
faculty = ''
for i in first_course + second_course + third_course + fourth_course:
  if i in faculties:
    faculty = i
  if i not in faculties:
    isu_data = pd.concat([isu_data, pd.DataFrame([[i.upper(), int(i[2]), faculty]], columns=isu_data.columns)])

In [ ]:
isu_data

,group,course,faculty
0,L3116,1,институт лазерных технологий
0,L3117,1,институт лазерных технологий
0,L3118,1,институт лазерных технологий
0,D3110,1,институт международного развития и партнерства
0,D3121C,1,институт перспективных систем передачи данных
...,...,...,...
0,B34003,4,центр прикладной оптики
0,B34222,4,центр химической инженерии
0,B34223,4,центр химической инженерии
0,B34232,4,центр химической инженерии


In [ ]:
isu_data.to_csv('/content/isu_data.csv')

In [ ]:
target_isu_data = pd.DataFrame(columns=['faculty', 'group', 'course', 'ISU_number', 'name'])

In [ ]:
group_urls = []

group_links = driver.find_elements(By.CLASS_NAME, "group-object")
print(group_links)

for link in group_links:
  group_urls.append(link.get_attribute("href"))

for group_url in group_urls:
    driver.get(group_url)
    WebDriverWait(driver, 10).until(EC.url_to_be(group_url))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    fio_elements = soup.find_all('td', class_='report_column', headers='ФИО')
    id_elements = soup.find_all('td', class_='report_column', headers='ИД')

    breadcrumb_element = soup.find('ul', class_='breadcrumb bg-itmo-neutral')
    span_element = breadcrumb_element.find('li', class_='active').find('span')
    group_number = span_element.text.strip()

    if group_number in page_contents_upper:
        fio_list = [element.a.text.strip() for element in fio_elements]
        id_list = [element.text.strip() for element in id_elements]

        for fio, id_number in zip(fio_list, id_list):
            print(isu_data.query('group == @group_number')['faculty'].to_string()[1:].strip())
            target_isu_data = pd.concat(
                [target_isu_data, pd.DataFrame([[isu_data.query('group == @group_number')['faculty'].to_string()[1:].strip(),
                                                group_number,
                                                isu_data.query('group == @group_number')['course'].to_string()[1:].strip(),
                                                id_number,
                                                fio]], columns=target_isu_data.columns)])

    else:
        break

    driver.back()

In [ ]:
target_isu_data.to_csv('/content/target_isu_data.csv')

Дальше я пользовалась сохранёнными csv-файлами, чтобы не выполнять скрейпинг каждый раз.

In [ ]:
isu_data = pd.read_csv('/content/isu_data.csv')
target_isu_data = pd.read_csv('/content/target_isu_data.csv')

In [ ]:
target_faculty_data = target_isu_data.query('faculty == @target_faculty_name').drop(columns='Unnamed: 0')
target_faculty_data.head()

,faculty,group,course,ISU_number,name
532,факультет инфокоммуникационных технологий,K3120,1,375250,Арройо Ариас Николас Рафаэль
533,факультет инфокоммуникационных технологий,K3120,1,408374,Вечеренко Анастасия Дмитриевна
534,факультет инфокоммуникационных технологий,K3120,1,408637,Закоурцев Андрей Вадимович
535,факультет инфокоммуникационных технологий,K3120,1,408655,Зенин Данил Дмитриевич
536,факультет инфокоммуникационных технологий,K3120,1,408672,Зубов Алексей Андреевичч


#### Ответы на вопросы

1. Одинаковое ли количество групп от курса к курсу?

In [ ]:
isu_data.query('faculty == @target_faculty_name')[['group', 'course']].value_counts('course')

course
1    11
2    10
3    10
4     9
dtype: int64

Количество групп на курсах не одинаково.

2. Есть ли в вашей выборке однофамильцы (для простоты учитывайте только полное совпадение фамилии)? Если да, выведите этих людей.

In [ ]:
def find_duplicates(group):
    if len(group) > 1:
        return group

to_group = [i.split()[0] for i in target_faculty_data['name'].to_list()]

target_faculty_data.groupby(to_group).apply(find_duplicates)

faculty   group  course  \
Акулов    3477  факультет инфокоммуникационных технологий   K3239       2   
          5939  факультет инфокоммуникационных технологий  K33391       3   
Алексеев  731   факультет инфокоммуникационных технологий   K3144       1   
          3396  факультет инфокоммуникационных технологий   K3221       2   
          7928  факультет инфокоммуникационных технологий  K34421       4   
...                                                   ...     ...     ...   
Шаповалов 694   факультет инфокоммуникационных технологий   K3141       1   
          3509  факультет инфокоммуникационных технологий   K3239       2   
Шевченко  749   факультет инфокоммуникационных технологий   K3144       1   
          3390  факультет инфокоммуникационных технологий   K3220       2   
          7889  факультет инфокоммуникационных технологий  K34401       4   

                ISU_number                          name  
Акулов    3477      373332      Акулов Даниил Даниилович  
          5939      311593   Акулов Алексей Владимирович  
Алексеев  731       408121    Алексеев Николай Сергеевич  
          3396      367800      Алексеев Тимофей Юрьевич  
          7928      313240     Алексеев Павел Алексеевич  
...                    ...                           ...  
Шаповалов 694       409874   Шаповалов Сергей Кириллович  
          3509      371999      Шаповалов Илья Андреевич  
Шевченко  749       409886       Шевченко Семён Олегович  
          3390      369054  Шевченко Кристина Валерьевна  
          7889      313265    Шевченко Максим Дмитриевич  

[135 rows x 5 columns]

Однофамильцы в выборке есть.

3. Есть ли в вашей выборке ребята без отчества? Если да, выведите этих людей.

In [ ]:
people_without_patronymic = target_faculty_data[target_faculty_data['name'].str.split(' ').apply(len) == 2]

In [ ]:
people_without_patronymic

,faculty,group,course,ISU_number,name
560,факультет инфокоммуникационных технологий,K3121,1,372463,Маатук Айхам
689,факультет инфокоммуникационных технологий,K3141,1,413091,Тутубалин Кирилл
3385,факультет инфокоммуникационных технологий,K3220,2,368947,Уразалин Амир
3460,факультет инфокоммуникационных технологий,K3223,2,336094,Жбейл Жеорж
3578,факультет инфокоммуникационных технологий,K3241,2,373765,Шмидт Арсений
3605,факультет инфокоммуникационных технологий,K3243,2,368572,Невзорова Вероника
5878,факультет инфокоммуникационных технологий,K33201,3,336755,Юсупов Данил
5910,факультет инфокоммуникационных технологий,K33211,3,336743,Назыров Эмиль
5946,факультет инфокоммуникационных технологий,K33391,3,336101,Ким Даниил
5960,факультет инфокоммуникационных технологий,K33392,3,336100,Байков Иван


Люди без отчества есть.

4. Найдите курс, на котором меньше всего обучающихся. Выведите значение и номер курса. Если таких курсов несколько, выведите их все.

In [ ]:
course_counts = target_faculty_data.value_counts('course').sort_values(ascending=True)

In [ ]:
course_counts = course_counts.to_string().split()[1:]

In [ ]:
print(f"Наименьшее количество обучающихся - {course_counts[1]}, на {course_counts[0]} курсе")

Наименьшее количество обучающихся - 151, на 4 курсе


5. Для каждого курса найдите группу, в которой больше всего обучающихся. Выведите тройки (курс, группа, кол-во обучающихся). Если для какого-либо курса таких групп окажется несколько, выведите их все.

In [ ]:
grouped_data = target_faculty_data.groupby(['course', 'group']).size()

In [ ]:
max_student_groups_idx = grouped_data.groupby(level=0).idxmax()

In [ ]:
print(max_student_groups_idx)

course
1     (1, K3140)
2     (2, K3241)
3    (3, K33201)
4    (4, K34211)
dtype: object


In [ ]:
for i in max_student_groups_idx:
  print(f"Курс: {i[0]}, группа: {i[1]}, количество обучающихся: {target_faculty_data.query('group == @i[1]')['ISU_number'].count()}")

Курс: 1, группа: K3140, количество обучающихся: 34
Курс: 2, группа: K3241, количество обучающихся: 36
Курс: 3, группа: K33201, количество обучающихся: 28
Курс: 4, группа: K34211, количество обучающихся: 23


### Задание 2

Одни из самых ужасных с точки зрения формата представления данные - это данные на сайтах государственных организаций (поликлиники, школы и т.п.). Потренируемся работать с подобными сайтами. Соберем данные за муниципальный этап Всероссийской олимпиады школьников [2022/2023 года по Томской области](http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/).

БОльшая часть кода уже написана за вас, однако вам необходимо дополнить функционал по непосредственному извлечению pandas-датафрейма из файлов различного формата (`pdf`, `xlsx`, `docx`), а также посчитать статистические показатели по собранным данным.

Изучите данные на сайте - таблицы имеют схожую, хоть и слегка различающуюся, структуру. Приведем извлеченную из каждого файла таблицу к датафрейму с колонками `предмет`, `класс`, `фио`, `баллы`. Если что-либо отсутствует - заполним пустой строкой, если речь идёт не о баллах, иначе нулем. ФИО можно расматривать как склейку через пробел полей "Фамилия", "Имя" и "Отчество" в случае наличия последнего.

Обратите внимание, что вы можете менять любую часть кода, добавлять свои функции, а также произвести парсинг полностью вручную (хотя это нежелательно).

In [ ]:
!pip install python-docx xls2xlsx img2table[easyocr]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.5/398.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: pol

In [ ]:
import numpy as np

In [ ]:
from img2table.document import PDF
from img2table.ocr import EasyOCR
from xls2xlsx import XLS2XLSX
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from docx import Document


easyocr = EasyOCR(lang=["ru"], kw={"gpu": False})  # можно использовать любой другой OCR-движок

Из любого файла  нам нужно достать только ФИО (или фамилию и имя) и баллы, всё остальное можно даже не сохранять.
Нужные колонки могут называться Фамилия, Имя, Отчество, Результат, Балл (причём там, где нет колонки с отчеством, колонка с баллами называется "Балл", иначе - "Результат")

- Во всех excel-файлах в этой табличке первые две строки нужно удалить, а колонки переименовать.
- В pdf-файлах каждая страница распознаётся как отдельная таблица. Их нужно слепить вместе и переименовать колонки.
- В docx-файлах нужно только переименовать колонки.

In [ ]:
def get_file_extension(filename):
    ext = filename.split('.')[-1]
    if ext not in ['xls', 'xlsx', 'pdf', 'docx']:
        raise ValueError
    return ext


def download_file(url, destination):
    response = requests.get(url)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded successfully to '{destination}'")
    else:
        print(f"Failed to download file from '{url}'")


def convert_xls_to_xlsx(filename):  # конвертация xlsx-файлов устаревшего образца
    x2x = XLS2XLSX(filename)
    x2x.to_xlsx(filename + "x")
    return filename


def extract_data(subject, grade, filename, ext, verbose=True):
    if ext == 'xlx':
        ext = 'xlsx'
        filename = convert_xls_to_xlsx(filename)

    if ext == 'xlsx':
        df = parse_xlsx(filename)
    elif ext == 'pdf':
        df = parse_pdf(filename)
    elif ext == 'docx':
        df = parse_docx(filename)

    if "Отчество" in df.columns:
        df = df.rename(columns={
            "Результ\nат" : "Результат"
        })
        df_first = df[['Фамилия', 'Имя', 'Отчество', 'Результат']]
        df_second = pd.DataFrame(columns=['предмет', 'класс', 'фио', 'баллы'])
        for i in range(df_first.shape[0]):
            df_second = pd.concat([df_second, pd.DataFrame([[
                subject,
                grade,
                str(df_first['Фамилия'].iloc[i]).replace('\n', '') +
                ' ' + str(df_first['Имя'].iloc[i]).replace('\n', '') +
                ' ' + str(df_first['Отчество'].iloc[i]).replace('None', '').replace('nan', '').replace('\n', ''),
                str(df_first['Результат'].iloc[i]).replace(',', '.')
            ]], columns=df_second.columns)], ignore_index=True)
    else:
        df_first = df[['Фамилия', 'Имя', 'Балл']]
        df_second = pd.DataFrame(columns=['предмет', 'класс', 'фио', 'баллы'])
        for i in range(df.shape[0]):
            df_second = pd.concat([df_second, pd.DataFrame([[
                subject,
                grade,
                str(df_first['Фамилия'].iloc[i]) + ' ' + str(df_first['Имя'].iloc[i]),
                str(df_first['Балл'].iloc[i]).replace(',', '.')
            ]], columns=df_second.columns)], ignore_index=True)

    if verbose:
        print()
        display(df_second.head())
        print()

    return df_second


def parse_xlsx(filename):
    data = pd.read_excel(filename)
    new_data = pd.DataFrame(columns=data.iloc[2, :].values.tolist())
    new_columns = {}
    for i, col in enumerate(new_data.columns):
      new_columns[data.columns[i]] = col

    return data.rename(columns=new_columns)[3:].reset_index(drop=True)


def parse_docx(filename):
    document = Document(filename)
    data = pd.DataFrame()

    table = document.tables[0]

    ls = []
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                ls.append(paragraph.text)

    data = pd.DataFrame(np.array(ls).reshape(len(table.rows), len(table.columns)))
    new_columns = {}
    new_data = pd.DataFrame(columns=data.iloc[0, :].values.tolist())
    for i, col in enumerate(new_data.columns):
      new_columns[data.columns[i]] = col

    return data.rename(columns=new_columns)[1:].reset_index(drop=True)


def parse_pdf(filename):
    pdf = PDF(filename,
          detect_rotation=False,
          pdf_text_extraction=True)

    extracted_tables = pdf.extract_tables(ocr=easyocr,
                                      implicit_rows=False,
                                      borderless_tables=False,
                                      min_confidence=50)
    data = pd.DataFrame(columns=extracted_tables[0][0].df.columns)
    for i in range(len(extracted_tables)):
        data = pd.concat([data, extracted_tables[i][0].df])

    new_columns = {}
    new_data = pd.DataFrame(columns=data.iloc[0, :].values.tolist())
    for i, col in enumerate(new_data.columns):
      new_columns[data.columns[i]] = col

    return data.rename(columns=new_columns)[1:].reset_index(drop=True)

In [ ]:
df_1 = parse_xlsx('/content/9-klass-e-konomika.xlsx')

In [ ]:
df_1.head()

,№,Фамилия,Имя,Образовательная организация,Класс обучения,Балл,Муниципалитет
0,1,Зюбин,Владислав,МАОУ СОШ № 32 г. Томска,8,84,город Томск
1,2,Кириллов,Иван,Муниципальное бюджетное общеобразовательное уч...,9,74,ЗАТО Северск
2,3,Телицин,Артемий,Муниципальное бюджетное общеобразовательное уч...,9,74,ЗАТО Северск
3,4,Шарабарина,Елизавета,МАОУ «СОШ №7 им. Н.Г.Барышева»,9,72,Колпашевский район
4,5,Кочетов,Дмитрий,Муниципальное бюджетное общеобразовательное уч...,9,67,ЗАТО Северск


In [ ]:
df_2 = parse_docx('/content/Astronomiya-Rejting-7-klass.docx')

In [ ]:
df_2.head()

,№ п/п,Фамилия,Имя,Отчество,Наименование ОО,Класс обучения,Результат,"Муниципалитет (округ), город",% от максимально возможного балла
0,,Харин,Ростислав,Евгеньевич,ОГБОУ «ТФТЛ»,7,5,Город Томск,"10,42%"
1,,Скороходов,Юрий,Сергеевич,МАОУ гимназия № 6 г.Томска,7,4,Город Томск,"8,33%"
2,,Степанов,Роман,Дмитриевич,Муниципальное автономное общеобразовательное у...,7,4,Асиновский,"8,33%"
3,,Волкова,Дарья,Алексеевна,Муниципальное автономное общеобразовательное у...,7,4,Асиновский,"8,33%"
4,,Овцина,Арина,Алексеевна,МАОУ СФМЛ,7,4,ЗАТО Северск,"8,33%"


In [ ]:
df_3 = parse_pdf('/content/E-kologiya-Rejting-10-klass.pdf')

В обработке PDF присутствует костыль - OCR считывает каждую страницу как отдельную таблицу, из-за чего в последних таблицах отсутствует колонка "является победителем/призёром прошлого года", так как все значения в ней пустые. В результате значения из колонки "Муниципалитет" переносятся в колонку про призёрство прошлого года.

Я не стала это менять, потому что этими данными в рамках этой работы я не пользуюсь, но это очень некрасиво.

In [ ]:
df_3.head()

,№ п/п,Фамилия,Имя,Наименование ОО,Класс\nобучения,Результат,Является\nпобедителем/\nпризером РЭ\nВсОШ\n2021/22 уч..г.,"Муниципалитет\n(округ), город"
0,1.,Шашко,Максим,Муниципальное общеобразовательное учреждение «...,10,47,None,Стрежевой
1,2.,Норкина,Элеонора,Областное государственное автономное общеобраз...,10,43,призёр,Город Томск
2,3.,Демченко,Дарья,Муниципальное бюджетное общеобразовательное уч...,10,39,None,Город Томск
3,4.,Ясинская,Полина,Муниципальное автономное общеобразовательное у...,10,38,None,Город Томск
4,5.,Михайлова,Ольга,Муниципальное бюджетное общеобразовательное уч...,10,"36,5",None,ЗАТО Северск


Для простоты будем работать только со следующим списком предметов: экономика, английский язык, математика, биология, информатика и ИКТ. Следующий код менять не нужно.

In [ ]:
target_subjects = ['Экономика', 'Английский язык', 'Математика', 'Биология', 'Информатика и ИКТ']  # рассмотрим только эти предметы

all_dfs = []
response = requests.get('http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/')  # результаты муниципального этапа за 2022-2023 год
main_table = BeautifulSoup(response.text).find('table').findAll('table')[2]  # основная таблица (см. сайт)

for row in main_table.findAll('tr')[1:]:  # первая строка - шапка таблицы
    cells = row.findAll('td')

    subject = cells[0].text.strip()  # название предмета из таблицы
    if subject not in target_subjects:
        continue

    a_hrefs = cells[-1].findAll('a')
    for a_href in a_hrefs:  # ссылки на файлы
        grade = re.search(r'\d+', a_href.text).group(0)
        href = a_href.get('href')

        ext = get_file_extension(href)
        filename = f"{subject}_{grade}.{ext}"
        download_file(href, filename)  # скачивание файла

        extracted_df = extract_data(subject, grade, filename, ext, verbose=True)  # достать данные из файла
        all_dfs.append(extracted_df)

overall_df = pd.concat(all_dfs, ignore_index=True)
overall_df.loc[overall_df['баллы'] == '', 'баллы'] = 0
overall_df.loc[overall_df['баллы'] == 'None', 'баллы'] = 0  # фикс для NaN-ов в случае баллов
overall_df['баллы'] = overall_df['баллы'].replace(',', '.').astype(float)  # конвертация баллов в числовой формат

File downloaded successfully to 'Экономика_9.xlsx'



,предмет,класс,фио,баллы
0,Экономика,9,Зюбин Владислав,84
1,Экономика,9,Кириллов Иван,74
2,Экономика,9,Телицин Артемий,74
3,Экономика,9,Шарабарина Елизавета,72
4,Экономика,9,Кочетов Дмитрий,67



File downloaded successfully to 'Экономика_10.pdf'



,предмет,класс,фио,баллы
0,Экономика,10,Соколов Владислав Андреевич,71
1,Экономика,10,Танасейчук Тимофей Владимирович,67
2,Экономика,10,Шашко Максим Дмитриевич,64
3,Экономика,10,Аникин Лев Дмитриевич,63
4,Экономика,10,Сухоруков Николай Максимович,61



File downloaded successfully to 'Экономика_11.pdf'



...

,предмет,класс,фио,баллы
0,Информатика и ИКТ,9,Лисок Артем Александрович,381
1,Информатика и ИКТ,9,Лисок Никита Александрович,329
2,Информатика и ИКТ,9,Александров Илья Александрович,303
3,Информатика и ИКТ,9,Ивасенко Илья Данилович,255
4,Информатика и ИКТ,9,Полубоярцев Данил Дмитриевич,252



File downloaded successfully to 'Информатика и ИКТ_10.pdf'



,предмет,класс,фио,баллы
0,Информатика и ИКТ,10,Осипов Вячеслав Витальевич,388
1,Информатика и ИКТ,10,Атимасов Никита Игоревич,286
2,Информатика и ИКТ,10,Ильин Василий Игоревич,274
3,Информатика и ИКТ,10,Будаев Никита Олегович,255
4,Информатика и ИКТ,10,Соколов Владислав Андреевич,250



File downloaded successfully to 'Информатика и ИКТ_11.pdf'



,предмет,класс,фио,баллы
0,Информатика и ИКТ,11,Агафонов Артём Александрович,500
1,Информатика и ИКТ,11,Зюбин Константин Сергеевич,500
2,Информатика и ИКТ,11,Зиновьев Алексей Игоревич,461
3,Информатика и ИКТ,11,Разинков Иван Константинович,429
4,Информатика и ИКТ,11,Фокин Степан Дмитриевич,429


Должно получиться что-то похожее:

In [ ]:
overall_df

,предмет,класс,фио,баллы
0,Экономика,9,Зюбин Владислав,84.0
1,Экономика,9,Кириллов Иван,74.0
2,Экономика,9,Телицин Артемий,74.0
3,Экономика,9,Шарабарина Елизавета,72.0
4,Экономика,9,Кочетов Дмитрий,67.0
...,...,...,...,...
2051,Информатика и ИКТ,11,Писаров Александр Михайлович,81.0
2052,Информатика и ИКТ,11,Фрейдман Павел Александрович,80.0
2053,Информатика и ИКТ,11,Николаева Анастасия Александровна,70.0
2054,Информатика и ИКТ,11,Матвеева Юлия Алексеевна,9.0


Постройте сводную таблицу - по каждому предмету для каждого класса посчитайте минимальный балл, средний балл, медианный балл, а также максимальный балл.

Посмотрите на результаты по информатике. О чём могут свидетельствовать подобные цифры?

In [ ]:
overall_df['баллы'] = overall_df['баллы'].apply(lambda val: float(str(val).replace(',', '.')))  # конвертация баллов в числовой формат

In [ ]:
info = pd.DataFrame()

In [ ]:
info = overall_df.groupby(['предмет', 'класс']).count()

In [ ]:
info.head()

фио  баллы
предмет         класс            
Английский язык 10     156    156
                11     180    180
                9      168    168
Биология        10     128    128
                11     139    139

In [ ]:
info['max'] = overall_df.groupby(['предмет', 'класс']).max()['баллы']
info['min'] = overall_df.groupby(['предмет', 'класс']).min()['баллы']
info['mean'] = overall_df.groupby(['предмет', 'класс']).mean()['баллы']
info['median'] = overall_df.groupby(['предмет', 'класс']).median()['баллы']

<ipython-input-101-277a02cb5807>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  info['mean'] = overall_df.groupby(['предмет', 'класс']).mean()['баллы']
<ipython-input-101-277a02cb5807>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  info['median'] = overall_df.groupby(['предмет', 'класс']).median()['баллы']


In [ ]:
info.drop(columns=['фио']).rename(columns={
    'баллы' : 'count'
})

count    max  min        mean  median
предмет           класс                                       
Английский язык   10       156   54.0  0.0   24.628205    24.0
                  11       180   46.0  0.0   26.286111    28.0
                  9        168   45.0  1.0   21.319643    23.0
Биология          10       128   66.0  0.0   31.925000    31.0
                  11       139   71.0  0.0   37.262230    36.2
                  9        220   55.0  7.0   24.850227    25.0
Информатика и ИКТ 10        59  388.0  0.0  139.406780   123.0
                  11        53  500.0  0.0  202.830189   168.0
                  9         36  381.0  0.0  126.861111   110.5
Математика        10       171   26.0  0.0    4.201754     1.0
                  11       206   28.0  0.0    6.160194     7.0
                  8        109   28.0  0.0    2.903670     2.0
                  9        165   35.0  0.0    6.703030     7.0
Экономика         10        84   71.0  1.0   26.863095    23.0
                  11        99   69.0  4.0   32.058586    32.0
                  9         83   84.0  0.0   28.518072    23.0

По информатике у всех участников баллы намного выше, чем по всем остальным предметам. Это связано с тем, что задания по информатике стоят "дороже": за одну задачу, например, можно получить 100 баллов, когда по другим предметам количество баллов за задание обычно однозначное или чуть больше 10.